In [11]:
import pygame
import random
import time

In [22]:
# 1. 게임 초기화
pygame.init()


# 2. 게임창 옵션 설정
size = [300, 600]
screen = pygame.display.set_mode(size)

title = "My Game"
pygame.display.set_caption(title)


# 3. 게임 내 필요한 설정
clock = pygame.time.Clock()

class obj:
    def __init__(self):
        self.x = 0
        self.y = 0 
        self.move = 0
    # 이미지 불러오기
    def put_img(self, address):
        if address[-3:] == "png":
            self.img = pygame.image.load(address).convert_alpha()
        else:
            self.img = pygame.image.load(address)
        self.sx, self.sy = self.img.get_size()
    # 이미지 사이즈 바꾸기
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img, (sx, sy))
        self.sx, self.sy = self.img.get_size()
    # 이미지를 배경에 띄우기
    def show(self):
        screen.blit(self.img, (self.x, self.y))

# 충돌 설정을 위한 함수
def crash(a, b):
    if (a.x - b.sx <= b.x) and (b.x <= a.x + a.sx):
        if (a.y - b.sy <= b.y) and (b.y <= a.y + a.sy):
            return True
        else: 
            return False
    else: 
        return False

rocket_img = obj()
rocket_img.put_img("D:/temp/rocket.png")
rocket_img.change_size(100, 100)

# 이미지 띄울 위치 변경
rocket_img.x = round(size[0]/2) - round(rocket_img.sx/2)
rocket_img.y = size[1] - rocket_img.sy-15

# 방향키를 움직일 때마다 로켓 이미지가 얼마나 움직일지를 결정하는 변수
rocket_img.move = 20

# 방향키를 눌렀을 때 이미지가 움직일지 말지를 결정하는 변수
left_go = False
right_go = False
space_go = False

# 총알 리스트
bullet_list = []
# 외계인 리스트
alien_list = []

# 총알 연사 속도 조절 변수
k = 0

black = (0, 0, 0)
white = (255, 255, 255)



# 4. 메인 이벤트
SB = 0
while SB == 0:

    # 4-1. FPS 설정
    clock.tick(30)  # 1초에 30번 while문이 돎

    # 4-2. 각종 입력 감지
    for event in pygame.event.get():  # 어떤 이벤트를 받아서 하나씩 대조
        if event.type == pygame.QUIT:
            SB = 1
        # 어떤 키를 눌렀을 때
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:  # 왼쪽 방향키
                left_go = True
            elif event.key == pygame.K_RIGHT:  # 오른쪽 방향키
                right_go = True
            elif event.key == pygame.K_SPACE:  # 스페이스바
                space_go = True
                k = 0  # 스페이스바를 연타할 때도 총알 발사
        # 어떤 키를 뗐을 때
        elif event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                left_go = False
            elif event.key == pygame.K_RIGHT:
                right_go = False
            elif event.key == pygame.K_SPACE:
                space_go = False
            
    # 4-3. 입력, 시간에 따른 변화
    if left_go == True:
        rocket_img.x -= rocket_img.move
        if rocket_img.x <= 0:
            rocket_img.x = 0
    elif right_go == True:
        rocket_img.x += rocket_img.move
        if rocket_img.x >= size[0] - rocket_img.sx:
            rocket_img.x = size[0] - rocket_img.sx
    
    # 스페이스바를 눌렀을 때 총알 처리
    if space_go == True and k % 6 == 0:  # 총알이 나오는 속도 늦추기
        bullet_img = obj()
        bullet_img.put_img("D:/temp/bullet.png")
        bullet_img.change_size(10, 15)
        bullet_img.x = round(rocket_img.x + rocket_img.sx/2 - bullet_img.sx/2)
        bullet_img.y = rocket_img.y - bullet_img.sy
        bullet_img.move = 15
        bullet_list.append(bullet_img)
    k += 1  # 총알이 나오는 속도 늦추기
        
    # 이때까지 생성한 총알들의 움직임 처리
    d_list = []  # 화면 밖으로 나간 총알 리스트
    for i in range(len(bullet_list)):
        bullet = bullet_list[i]
        bullet.y -= bullet.move
        if bullet.y <= -bullet.sy:
            d_list.append(i)
    # 화면 밖으로 나간 총알 삭제
    for d in d_list:
        del bullet_list[d]
        
    # 외계인 이미지를 랜덤으로
    if random.random() > 0.98:
        alien_img = obj()
        alien_img.put_img("D:/temp/alien.png")
        alien_img.change_size(30, 30)
        alien_img.x = random.randrange(0, size[0]-alien_img.sx-round(rocket_img.sx/2))
        alien_img.y = 10
        alien_img.move = 1
        alien_list.append(alien_img)
    
    # 외계인 이미지 처리
    d_list2 = []
    for i in range(len(alien_list)):
        alien = alien_list[i]
        alien.y += alien.move
        if alien.y >= size[1]:
            d_list2.append(i)
    # 화면 밖으로 나간 외계인 삭제
    for d in d_list2:
        del alien_list[d]
        
    # 외계인과 충돌한 총알 처리
    db_list = []
    da_list = []
    
    # 총알과 외계인을 하나씩 리스트에서 꺼내어
    for i in range(len(bullet_list)):
        for j in range(len(alien_list)):
            bullet = bullet_list[i]
            alien = alien_list[j]
            # 위치를 비교한 후 삭제 리스트에 추가
            if crash(bullet, alien) == True:
                db_list.append(i)
                da_list.append(j)
    # 중복 방지(리스트를 집합으로 변환)
    db_list = list(set(db_list))
    da_list = list(set(da_list))
    
    # list index out of range 방지
    try:
        for db in db_list:
            del bullet_list[db]
        for da in da_list:
            del alien_list[da]
    except:
        pass
        
    # 외계인과 로켓이 충돌했을 경우 게임 종료
    for i in range(len(alien_list)):
        alien = alien_list[i]
        if crash(alien, rocket_img) == True:
            SB = 1
            time.sleep(1)
            
    
    # 4-4. 그리기
    screen.fill(black)   # 화면 색깔 까맣게 지정
    # screen.blit(rocket_img, (rocket_img_x,ro cket_img_y))  # 로켓 이미지 불러오기
    rocket_img.show()
    # 총알 이미지 불러오기(이때까지 스페이스바를 눌러서 생성한 총알 모두를 보여줌)
    for bullet in bullet_list:
        bullet.show()
    # 외계인 이미지 불러오기
    for alien in alien_list:
        alien.show()
    
    # 4-5. 업데이트
    pygame.display.flip()

# 5. 종료
pygame.quit()